### The repository is processing rate map from papers to fit requirement of stdpopsim 

The exact format refer to [msprime.RateMap.read_hapmap](https://tskit.dev/msprime/docs/stable/api.html#msprime.RateMap.read_hapmap) 

#### Johnsson_2021

Johnsson_2021 use the data from the supplyment of [paper](https://doi.org/10.1186/s12711-021-00643-0) 

In additional file has sex-averaged (Additional file 4: Table S3),male-specific (Additional file 2: Table S1), 

and female-specific (Additional file 3: Table S2) rate map. Here, we use the value of sex-averaged rate map.

In [2]:
import msprime
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv("./Johnsson_2021/Sex-averaged map of the landscape of pig recombination rate in 1-Mb windows.csv", sep=",", header=0)

In [11]:
df

,chr,window_start,window_end,recombination_rate
0,1,1,1000000,0.009154
1,1,1000001,2000000,0.009496
2,1,2000001,3000000,0.006806
3,1,3000001,4000000,0.011100
4,1,4000001,5000000,0.015028
...,...,...,...,...
2267,18,51000001,52000000,0.018101
2268,18,52000001,53000000,0.010463
2269,18,53000001,54000000,0.008005
2270,18,54000001,55000000,0.010662


In [12]:
# Avergae recombination rate for each chromosome
li_rate_mean = []
li_length=[]
for i in range(1,19):
    chr_data=df.groupby('chr').get_group(i)
    r=list(chr_data["recombination_rate"]*1e-6)
    p=[0]
    for j in chr_data["window_end"]:
        p.append(j)

    rate_map = msprime.RateMap(
    position=p,
    rate=r
    )
    li_length.append(rate_map.position[-1])
    li_rate_mean.append(rate_map.mean_rate)
li_rate_mean

[5.335247608983185e-09,
 8.6057260999254e-09,
 9.751141303348547e-09,
 9.79460049114614e-09,
 1.1992632296111e-08,
 8.786173521918499e-09,
 1.0887772285689678e-08,
 8.923031414594611e-09,
 9.337506174170301e-09,
 1.654233205589542e-08,
 1.2145120574910172e-08,
 1.6905062453467094e-08,
 6.162796630328349e-09,
 8.721569375548237e-09,
 8.114468762954893e-09,
 1.1407464686635653e-08,
 1.3779949516098884e-08,
 1.3679923888648167e-08]

In [16]:
# Calculate recombination rate and genetic distance for recombination map
for i in range(1,19):
    chr_data=df.groupby('chr').get_group(i)
    r=list(chr_data["recombination_rate"]*1e-6)
    p=[0]
    for j in chr_data["window_end"]:
        p.append(j)
    rate_map = msprime.RateMap(
    position=p,
    rate=r
    )
    # Calculate cumulative genetic distance (cM)
    genetic_positions = [0.0]
    position = [1]
    for j in range(len(rate_map.rate)):
        position.append(rate_map.position[j+1]+1)
        # Δp: distance between adjacent physical positions
        delta_p = rate_map.position[j+1] - rate_map.position[j]
        # Δg: genetic distance (in cM)
        delta_g = delta_p * rate_map.rate[j] * 100
        genetic_positions.append(genetic_positions[-1] + delta_g)

    # Create DataFrame for genetic map
    genetic_map_df=pd.DataFrame({
    "Chromosome": f'chr{i}',
    "Position(bp)": position[:-1],
    "Rate(cM/Mb)": np.array(rate_map.rate) * 1e8,  # Convert from M/bp to cM/Mb
    "Map(cM)": genetic_positions[:-1]
    })
    genetic_map_df.to_csv(f"./Johnsson_2021/Johnsson_2021_sex_average_rec_map/Johnsson_2021_Sex-averaged_map_chr{i}.txt",sep='\t', index=False)

In [17]:
!tar -cvf ./Johnsson_2021/Johnsson_2021_sex_average_rec_map.tar ./Johnsson_2021/Johnsson_2021_sex_average_rec_map
!gzip ./Johnsson_2021/Johnsson_2021_sex_average_rec_map.tar


a ./Johnsson_2021/Johnsson_2021_sex_average_rec_map
a ./Johnsson_2021/Johnsson_2021_sex_average_rec_map/Johnsson_2021_Sex-averaged_map_chr13.txt
a ./Johnsson_2021/Johnsson_2021_sex_average_rec_map/Johnsson_2021_Sex-averaged_map_chr7.txt
a ./Johnsson_2021/Johnsson_2021_sex_average_rec_map/Johnsson_2021_Sex-averaged_map_chr6.txt
a ./Johnsson_2021/Johnsson_2021_sex_average_rec_map/Johnsson_2021_Sex-averaged_map_chr12.txt
a ./Johnsson_2021/Johnsson_2021_sex_average_rec_map/Johnsson_2021_Sex-averaged_map_chr10.txt
a ./Johnsson_2021/Johnsson_2021_sex_average_rec_map/Johnsson_2021_Sex-averaged_map_chr4.txt
a ./Johnsson_2021/Johnsson_2021_sex_average_rec_map/Johnsson_2021_Sex-averaged_map_chr5.txt
a ./Johnsson_2021/Johnsson_2021_sex_average_rec_map/Johnsson_2021_Sex-averaged_map_chr11.txt
a ./Johnsson_2021/Johnsson_2021_sex_average_rec_map/Johnsson_2021_Sex-averaged_map_chr1.txt
a ./Johnsson_2021/Johnsson_2021_sex_average_rec_map/Johnsson_2021_Sex-averaged_map_chr15.txt
a ./Johnsson_2021/Johns

In [19]:
msprime.RateMap.read_hapmap("Johnsson_2021/Johnsson_2021/Johnsson_2021_sex_average_rec_map/Johnsson_2021_Sex-averaged_map_chr1.txt")

left,right,mid,span,rate
0,1,0.5,1,nan
1,1000001,500001,1000000,9.2e-09
1000001,2000001,1500001,1000000,9.5e-09
2000001,3000001,2500001,1000000,6.8e-09
3000001,4000001,3500001,1000000,1.1e-08
4000001,5000001,4500001,1000000,1.5e-08
5000001,6000001,5500001,1000000,3e-08
6000001,7000001,6500001,1000000,2.2e-08
7000001,8000001,7500001,1000000,1.5e-08
8000001,9000001,8500001,1000000,2e-08
